In [ ]:
import pandas as pd
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
def find_neighbors(df, p):
    x, y, z = p

    space_x = df['X'].unique()
    space_x.sort()
    space_x = np.mean(np.diff(space_x)[np.diff(space_x) > 1e-4])

    space_y = df['Y'].unique()
    space_y.sort()
    space_y = np.mean(np.diff(space_y)[np.diff(space_y) > 1e-4])

    space_z = df['Z'].unique()
    space_z.sort()
    space_z = np.mean(np.diff(space_z))

    space = {'X': space_x, 'Y': space_y, 'Z': space_z}

    # Helper function to find closest value
    def closest(df, axis, value):
        if axis == 'X':
            filtered = df[((df['Y'] - y).abs() < (space_y * 0.1)) & ((df['Z'] - z).abs() < (space_z * 0.1))]
        elif axis == 'Y':
            filtered = df[((df['X'] - x).abs() < (space_x * 0.1)) & ((df['Z'] - z).abs() < (space_z * 0.1))]
        else:  # axis == 'Z'
            filtered = df[((df['X'] - x).abs() < (space_x * 0.1)) & ((df['Y'] - y).abs() < (space_y * 0.1))]

        greater = filtered[filtered[axis] > value + space[axis] * 0.1]
        less = filtered[filtered[axis] < value - space[axis] * 0.1]
        closest_greater = greater.loc[greater[axis].sub(value).abs().idxmin()] if not greater.empty else None
        closest_lesser = less.loc[less[axis].sub(value).abs().idxmin()] if not less.empty else None
        return closest_greater, closest_lesser

    neighbors = {}
    for axis in ['X', 'Y', 'Z']:
        neighbors[f'greater_{axis}'], neighbors[f'lesser_{axis}'] = closest(df, axis, p[['X', 'Y', 'Z'].index(axis)])

    return pd.DataFrame.from_dict(neighbors).T

In [ ]:
cp_data = pd.read_parquet('./Data/data20231208/CP.parquet')
ic_bc_data = pd.read_parquet('./Data/data20231208/IC_BC.parquet')
all_data = pd.concat([cp_data, ic_bc_data])


all_position = all_data[['X', 'Y', 'Z']].drop_duplicates()
cp_position = cp_data[['X', 'Y', 'Z']].drop_duplicates()

In [ ]:
points = {                  # X-Y-Z
    'top_layer_cell' : [-0.02689803, -0.0151579, 0.0054],
    'top_layer_tab_1' : [-0.0445, 0.1185, 0.0054],
    'top_layer_tab_2' : [0.0445, 0.1185, 0.0054]
}

x_selected, y_selected, z_selected = points['top_layer_tab_2']
time = 90

In [ ]:
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

# First subplot with masked data
fig.add_trace(go.Scatter3d(x=cp_position['X'], y=cp_position['Y'], z=cp_position['Z'],
                           mode='markers',
                           marker=dict(size=3, opacity=0.8)),
              row=1, col=1)

# Second subplot with inverted mask data
fig.add_trace(go.Scatter3d(x=all_position['X'], y=all_position['Y'], z=all_position['Z'],
                           mode='markers',
                           marker=dict(size=3, opacity=0.7)),
              row=1, col=2)

fig.add_trace(go.Scatter3d(x=[x_selected], y=[y_selected], z=[z_selected],
                           mode='markers',
                           marker=dict(size=3, opacity=0.7, color='black')),
              row=1, col=2)

# Update layout for the entire figure
fig.update_layout(title="3D Scatter Plots with Masked and Inverted Mask Data",
                  scene=dict(xaxis_title='X Axis',
                             yaxis_title='Y Axis',
                             zaxis_title='Z Axis'),
                  scene2=dict(xaxis_title='X Axis',
                              yaxis_title='Y Axis',
                              zaxis_title='Z Axis'))

fig.show()

In [ ]:
res = find_neighbors(all_position, (x_selected, y_selected, z_selected))
res['name'] = res.index
current_df = all_data[
        (((all_data.X - x_selected)/x_selected).abs() < 1e-4)
        &
        (((all_data.Y - y_selected)/y_selected).abs() < 1e-4)
        &
        (((all_data.Z - z_selected)/z_selected).abs() < 1e-4)
        &
        (all_data.time == time)
    ][['X', 'Y', 'Z']]
current_df['name'] = 'current'
res = pd.concat([res, current_df])
none_res_df = res[res['X'].isna()]
res_df = res[~res['X'].isna()]
at_same_time = pd.merge(
    res_df, 
    all_data[all_data['time'] == time]
    , on=['X', 'Y', 'Z'], how='inner'
)
at_diff_time = pd.merge(
    all_data[all_data['time'].isin([ time-30, time+30])], current_df, on=['X', 'Y', 'Z'], how='inner'
)

final_df = pd.concat(
    [at_same_time, at_diff_time]
)

In [ ]:
temp_df = (
    pd.DataFrame(
        np.repeat((final_df[(final_df['name'] == 'current') & (final_df['time'] == time)]).values, none_res_df.shape[0], axis=0), columns=final_df.columns
    )
)
for i, value in enumerate(none_res_df['name'].values):
    str1, str2 = value.split("_")
    if str1 == 'greater':
        diff = (
            final_df[(final_df['name'] == 'current') & (final_df['time'] == time)][str2].values[0] - 
            final_df[(final_df['name'] == f'lesser_{str2}') & (final_df['time'] == time)][str2].values[0]
        )
    else:
        diff = -(
            final_df[(final_df['name'] == f'greater_{str2}') & (final_df['time'] == time)][str2].values[0] - 
            final_df[(final_df['name'] == 'current') & (final_df['time'] == time)][str2].values[0]
        )
    temp_df.loc[temp_df.index[i], 'name'] = value
    if str2 == 'X':
        X_new = x_selected + diff
        temp_df.loc[temp_df.index[i], 'X'] = X_new
    elif str2 == 'Y':
        Y_new = y_selected + diff
        temp_df.loc[temp_df.index[i], 'Y'] = Y_new
    else:
        Z_new = z_selected + diff
        temp_df.loc[temp_df.index[i], 'Z'] = Z_new

final_df = pd.concat([final_df, temp_df])

In [ ]:
# left

next_T = final_df[(final_df['name'] == 'current') & (final_df['time'] == time+30)]['Temp'].values[0]
pre_T = final_df[(final_df['name'] == 'current') & (final_df['time'] == time-30)]['Temp'].values[0]
heat_cur = final_df[(final_df['name'] == 'current') & (final_df['time'] == time)]['total_heat'].values[0]

dTdt = (next_T - pre_T)/60

# right
dx, dy, dz = [
    (final_df[final_df['name'] == f'greater_{axis}'][axis].values - final_df[final_df['name'] == f'lesser_{axis}'][axis].values)[0]/2
    for axis in ['X', 'Y', 'Z']
]

dTdx, dTdy, dTdz = [
    (final_df[final_df['name'] == f'greater_{axis}']['Temp'].values - final_df[final_df['name'] == f'lesser_{axis}']['Temp'].values)[0] /
    (final_df[final_df['name'] == f'greater_{axis}'][axis].values - final_df[final_df['name'] == f'lesser_{axis}'][axis].values)[0]
    for axis in ['X', 'Y', 'Z']
]

d2Tdx2, d2Tdy2, d2Tdz2 = [
    (
        (final_df[final_df['name'] == f'greater_{axis}']['Temp'].values - final_df[final_df['name'] == 'current']['Temp'].values)[0] /
        (final_df[final_df['name'] == f'greater_{axis}'][axis].values - final_df[final_df['name'] == 'current'][axis].values)[0] 
        -
        (final_df[final_df['name'] == 'current']['Temp'].values - final_df[final_df['name'] == f'lesser_{axis}']['Temp'].values)[0] /
        (final_df[final_df['name'] == 'current'][axis].values - final_df[final_df['name'] == f'lesser_{axis}'][axis].values)[0]
    ) / 
    ((final_df[final_df['name'] == f'greater_{axis}'][axis].values - final_df[final_df['name'] == f'lesser_{axis}'][axis].values)[0]/2)
    for axis in ['X', 'Y', 'Z']
]

# parameters
rho = 2092
C_p = 678
k = 18.2

# tab - al
rho = 2700
C_p = 900
k = 237

# tab - Cu
rho = 8960
C_p = 1460
k = 398

In [ ]:
# left
print('left side : rho*cp*dTdt : ', rho * C_p * dTdt)

# right
print('right side K grad ^2 T + Q: ', k * d2Tdx2 + k * d2Tdy2 + k * d2Tdz2 + heat_cur)